In [1]:
import pandas as pd
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 50)

In [2]:
start_year = 18
end_year = 19
weeks = [str(i).rjust(2,'0') for i in range(1,14)]
data_urls = []
while start_year < 24:
    for week in weeks:
        if start_year == 23 and week > '02':
            break
        if start_year == 19 and week == '04':
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-Delays-20{start_year}{end_year}-P{week}.zip')
        elif start_year == 19 and week > '04':
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-delays-20{start_year}-{end_year}-P{week}.zip')
        elif start_year > 19:
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-delays-20{start_year}-{end_year}-P{week}.zip')
        else:
            data_urls.append(f'https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/20{start_year}-{end_year}/All-Delays-20{start_year}-{end_year}-P{week}.zip')
    start_year+=1
    end_year+=1

In [3]:
cols_to_rename = {
    #New name: old name
    'APPLICABLE_TIMETABLE_FLAG': 'APP_TIMETABLE_FLAG_AFF',
    'FINANCIAL_YEAR_PERIOD': 'FINANCIAL_YEAR_AND_PERIOD',
    'INCIDENT_RESP_TRAIN': 'INCIDENT_RESPONSIBLE_TRAIN',
    'NR_LOCATION_MANAGER': 'NETWORK_RAIL_LOCATION_MANAGER',
    'EVENT_TYPE': 'PERFORMANCE_EVENT_CODE',
    'PLANNED_DEST_GBTT_DATETIME':'PLANNED_DEST_GBTT_DATETIME_AFF',
    'PLANNED_DEST_LOCATION_CODE':'PLANNED_DEST_LOC_CODE_AFFECTED',
    'PLANNED_DEST_WTT_DATETIME':'PLANNED_DEST_WTT_DATETIME_AFF',
    'PLANNED_ORIGIN_GBTT_DATETIME':'PLANNED_ORIG_GBTT_DATETIME_AFF',
    'PLANNED_ORIGIN_LOCATION_CODE':'PLANNED_ORIG_LOC_CODE_AFF',
    'PLANNED_ORIGIN_WTT_DATETIME':'PLANNED_ORIG_WTT_DATETIME_AFF',
    'REACT_REASON':'REACTIONARY_REASON_CODE',
    'SERVICE_GROUP_CODE':'SERVICE_GROUP_CODE_AFFECTED',
    'TIMING_LOAD':'TIMING_LOAD_AFFECTED',
    'TRACTION_TYPE': 'TRACTION_TYPE_AFFECTED',
    'TRAILING_LOAD': 'TRAILING_LOAD_AFFECTED',
    'TRAIN_SCHEDULE_TYPE' : 'TRAIN_SCHEDULE_TYPE_AFFECTED',
    'TRAIN_SERVICE_CODE': 'TRAIN_SERVICE_CODE_AFFECTED',
    'UNIT_CLASS':'UNIT_CLASS_AFFECTED',
    'TOC_CODE' : 'OPERATOR_AFFECTED',
    'TRUST_TRAIN_ID': 'TRUST_TRAIN_ID_AFFECTED',
    'RESP_MANAGER' : 'RESPONSIBLE_MANAGER',
    'TRUST_TRAIN_ID_REACT' : 'REACT_TRAIN',
    'TRUST_TRAIN_ID_RESP' : 'RESP_TRAIN'
    }

In [4]:
import numpy as np
stacked = pd.DataFrame()
for i in data_urls: # [0:13], [13:26], [26:39], [39:52], [52:65], [65:] for the six year-period
    df = pd.read_csv(i)
    columns = [col.upper() for col in list(df.columns)]
    df.columns = columns
    df.rename(columns = cols_to_rename, inplace = True)
    if df.iloc[-1]['ORIGIN_DEPARTURE_DATE'] == np.nan:
        df = df.iloc[:-1]
    df = df[df['OPERATOR_AFFECTED'] == 'EK']
#     df = df[(df['PERFORMANCE_EVENT_CODE'] == 'A') | (df['PERFORMANCE_EVENT_CODE'] == 'M')]
    
    df['ORIGIN_DEPARTURE_DATE'] = pd.to_datetime(df['ORIGIN_DEPARTURE_DATE'], format = '%d-%b-%y' if len(df.iloc[0]['ORIGIN_DEPARTURE_DATE'])==9 else '%d/%m/%Y %H:%M')
    df['PLANNED_ORIG_GBTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_GBTT_DATETIME_AFF'])
    df['PLANNED_ORIG_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_ORIG_WTT_DATETIME_AFF'])
    df['PLANNED_DEST_GBTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_GBTT_DATETIME_AFF'])
    df['PLANNED_DEST_WTT_DATETIME_AFF'] = pd.to_datetime(df['PLANNED_DEST_WTT_DATETIME_AFF'])
    df['INCIDENT_CREATE_DATE'] = pd.to_datetime(df['INCIDENT_CREATE_DATE'])
    df['INCIDENT_START_DATETIME'] = pd.to_datetime(df['INCIDENT_START_DATETIME'])
    df['INCIDENT_END_DATETIME'] = pd.to_datetime(df['INCIDENT_END_DATETIME'])
    df['EVENT_DATETIME'] = pd.to_datetime(df['EVENT_DATETIME'])
    df['TRAIN_SERVICE_CODE_AFFECTED'] = df['TRAIN_SERVICE_CODE_AFFECTED'].astype('int64', errors = 'ignore')
    df['UNIT_CLASS_AFFECTED'] = df['UNIT_CLASS_AFFECTED'].astype('str')
    if 'NON_PFPI_MINUTES' in df.columns:
        df['PFPI_MINUTES'] = df['PFPI_MINUTES'] + df['NON_PFPI_MINUTES']
        df = df.drop(columns = ['NON_PFPI_MINUTES'])

    #Takes care of pointless column in 2020/21 data
    if 'UNNAMED: 40' in df.columns:
        df = df.drop(columns = ['UNNAMED: 40'])

    if 'UNNAMED: 41' in df.columns:
        df = df.drop(columns = ['UNNAMED: 41'])
    print(f'Dataset: {i[-15:]} - shape is {df.shape}')
    stacked = pd.concat([stacked, df], ignore_index=True)

Dataset: 2018-19-P01.zip - shape is (14174, 39)
Dataset: 2018-19-P02.zip - shape is (13579, 39)
Dataset: 2018-19-P03.zip - shape is (12185, 39)
Dataset: 2018-19-P04.zip - shape is (13038, 39)
Dataset: 2018-19-P05.zip - shape is (13661, 39)
Dataset: 2018-19-P06.zip - shape is (12686, 39)
Dataset: 2018-19-P07.zip - shape is (14798, 39)
Dataset: 2018-19-P08.zip - shape is (17392, 39)
Dataset: 2018-19-P09.zip - shape is (20509, 39)
Dataset: 2018-19-P10.zip - shape is (22756, 39)
Dataset: 2018-19-P11.zip - shape is (16321, 39)
Dataset: 2018-19-P12.zip - shape is (17496, 39)
Dataset: 2018-19-P13.zip - shape is (18637, 39)
Dataset: 2019-20-P01.zip - shape is (16674, 39)
Dataset: 2019-20-P02.zip - shape is (11318, 39)
Dataset: 2019-20-P03.zip - shape is (11167, 39)
Dataset: -201920-P04.zip - shape is (13737, 39)
Dataset: 2019-20-P05.zip - shape is (14214, 39)
Dataset: 2019-20-P06.zip - shape is (13720, 39)
Dataset: 2019-20-P07.zip - shape is (13319, 39)
Dataset: 2019-20-P08.zip - shape is (146

/tmp/ipykernel_8414/1756658053.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Dataset: 2020-21-P12.zip - shape is (6352, 39)
Dataset: 2020-21-P13.zip - shape is (4748, 39)
Dataset: 2021-22-P01.zip - shape is (8035, 39)
Dataset: 2021-22-P02.zip - shape is (9637, 39)
Dataset: 2021-22-P03.zip - shape is (12021, 39)
Dataset: 2021-22-P04.zip - shape is (9597, 39)
Dataset: 2021-22-P05.zip - shape is (8181, 39)
Dataset: 2021-22-P06.zip - shape is (9784, 39)
Dataset: 2021-22-P07.zip - shape is (9318, 39)


/tmp/ipykernel_8414/1756658053.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Dataset: 2021-22-P08.zip - shape is (10203, 39)
Dataset: 2021-22-P09.zip - shape is (11244, 39)
Dataset: 2021-22-P10.zip - shape is (8617, 39)
Dataset: 2021-22-P11.zip - shape is (8662, 39)


...

Dataset: 2021-22-P13.zip - shape is (12254, 39)


/tmp/ipykernel_8414/1756658053.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Dataset: 2022-23-P01.zip - shape is (11287, 39)
Dataset: 2022-23-P02.zip - shape is (10309, 39)
Dataset: 2022-23-P03.zip - shape is (11726, 39)
Dataset: 2022-23-P04.zip - shape is (14979, 39)
Dataset: 2022-23-P05.zip - shape is (12909, 39)


/tmp/ipykernel_8414/1756658053.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Dataset: 2022-23-P06.zip - shape is (13870, 39)
Dataset: 2022-23-P07.zip - shape is (11285, 39)
Dataset: 2022-23-P08.zip - shape is (15051, 39)
Dataset: 2022-23-P09.zip - shape is (15365, 39)
Dataset: 2022-23-P10.zip - shape is (9631, 39)
Dataset: 2022-23-P11.zip - shape is (13395, 39)
Dataset: 2022-23-P12.zip - shape is (10365, 39)
Dataset: 2022-23-P13.zip - shape is (11347, 39)
Dataset: 2023-24-P01.zip - shape is (10362, 39)
Dataset: 2023-24-P02.zip - shape is (12940, 39)


In [5]:
stacked.to_csv(f'../data/EK.csv', mode = 'w', index = False)

In [6]:
stacked

,FINANCIAL_YEAR_AND_PERIOD,ORIGIN_DEPARTURE_DATE,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_LOC_CODE_AFF,PLANNED_ORIG_GBTT_DATETIME_AFF,PLANNED_ORIG_WTT_DATETIME_AFF,PLANNED_DEST_LOC_CODE_AFFECTED,PLANNED_DEST_GBTT_DATETIME_AFF,PLANNED_DEST_WTT_DATETIME_AFF,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,OPERATOR_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,TRAIN_SCHEDULE_TYPE_AFFECTED,TRACTION_TYPE_AFFECTED,TRAILING_LOAD_AFFECTED,TIMING_LOAD_AFFECTED,UNIT_CLASS_AFFECTED,INCIDENT_NUMBER,INCIDENT_CREATE_DATE,INCIDENT_START_DATETIME,INCIDENT_END_DATETIME,SECTION_CODE,NETWORK_RAIL_LOCATION_MANAGER,RESPONSIBLE_MANAGER,INCIDENT_REASON,ATTRIBUTION_STATUS,INCIDENT_EQUIPMENT,INCIDENT_DESCRIPTION,REACTIONARY_REASON_CODE,INCIDENT_RESPONSIBLE_TRAIN,PERFORMANCE_EVENT_CODE,START_STANOX,END_STANOX,EVENT_DATETIME,PFPI_MINUTES,RESP_TRAIN,REACT_TRAIN
0,2018/19_P01,2018-04-01,502V19MF01,50392.0,2018-04-01 09:54:00,2018-04-01 09:54:00,50401.0,2018-04-01 10:03:00,2018-04-01 10:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 09:54:00,0.0,NaN,NaN
1,2018/19_P01,2018-04-01,502V36MO01,50401.0,2018-04-01 14:11:00,2018-04-01 14:11:00,50392.0,2018-04-01 14:20:00,2018-04-01 14:20:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50401.0,50401.0,2018-04-01 14:11:00,0.0,NaN,NaN
2,2018/19_P01,2018-04-01,502V59MY01,50392.0,2018-04-01 19:54:00,2018-04-01 19:54:00,50401.0,2018-04-01 20:03:00,2018-04-01 20:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 19:54:00,0.0,NaN,NaN
3,2018/19_P01,2018-04-01,502V35MN01,50392.0,2018-04-01 13:54:00,2018-04-01 13:54:00,50401.0,2018-04-01 14:03:00,2018-04-01 14:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 13:54:00,0.0,NaN,NaN
4,2018/19_P01,2018-04-01,502V49MU01,50392.0,2018-04-01 17:24:00,2018-04-01 17:24:00,50401.0,2018-04-01 17:33:00,2018-04-01 17:33:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 17:24:00,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790298,2023/24_P02,2023-05-27,889F65M727,88392.0,2023-05-27 21:57:00,2023-05-27 21:57:00,52074.0,2023-05-27 22:19:00,2023-05-27 22:19:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,533095.0,2023-05-27,2023-05-27 17:42:00,2023-05-28 01:50:00,52084:87568,OQYA,VEK1,VA,Attribution Agreed,NaN,9C47 DISORDER ZLW ZCW,YI,9C47,C,88392.0,88392.0,2023-05-27 21:57:00,0.0,529C47MU27,529E64M727
790299,2023/24_P02,2023-05-27,889F67M827,88392.0,2023-05-27 22:27:00,2023-05-27 22:27:00,52074.0,2023-05-27 22:49:00,2023-05-27 22:49:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,533543.0,2023-05-27,2023-05-27 21:43:00,2023-05-28 05:54:00,87603:87643,OQM6,VEKA,VH,Attribution Agreed,NaN,9C62 PASS COMM NXG NWD,YD,9C62,M,88392.0,87568.0,2023-05-27 22:36:00,3.0,529C62MZ27,879D67M827
790300,2023/24_P02,2023-05-27,889F68M827,88392.0,2023-05-27 22:42:00,2023-05-27 22:42:00,52074.0,2023-05-27 23:04:00,2023-05-27 23:04:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,476572.0,2023-05-04,2023-04-30 00:01:00,2023-05-27 23:59:00,87643:87603,OQM6,QQA0,QA,Attribution Agreed,260124,BCY NXG ARL SCHEDULE PER

In [7]:
lookup_csv_path = "../mapping_data/stanox-lookup-lc.csv"
mapping_df = pd.read_csv(lookup_csv_path)
mapping_df

,Stanox,Latitude,Longitude,Station Name
0,50406,51.568070,0.129690,NaN
1,51556,51.580440,-0.072720,NaN
2,51560,51.580440,-0.072720,NaN
3,51603,51.544020,0.067840,EAST HAM DEP COUNTRY E. JN
4,51725,51.747420,-0.010500,BROXBOURNE DOWN PASS LOOP
...,...,...,...,...
9719,87601,51.505064,-0.086064,London Bridge Rail Station
9720,87214,51.486189,-0.122857,Vauxhall Rail Station
9721,87201,51.495257,-0.144527,London Victoria Rail Station
9722,86005,51.361768,-0.457711,Weybridge Rail Station


In [8]:
origin_geo_locations = pd.merge(stacked,
                        mapping_df,
                        left_on ='START_STANOX',
                        right_on = 'Stanox' ,
                        how ='left')
    # drop observation w/o origin latitude and longitude, drop redundant columns
# origin_geo_locations = origin_geo_locations.dropna(subset=['Latitude', 'Longitude'])
origin_geo_locations = origin_geo_locations.drop(columns = ['Stanox'])
new_column_names = {'Latitude': 'Lat_OR', 'Longitude': 'Lon_OR', 'Station Name': 'STATION_OR'}
origin_geo_locations.rename(columns=new_column_names, inplace=True)

# merge mapping dataset with the train delays data to get latitude and longitude for destination stanox code
origin_dest_locations = pd.merge(origin_geo_locations,
                  mapping_df,
                  left_on ='END_STANOX',
                  right_on = 'Stanox' ,
                  how ='left')
# drop observation w/o destination latitude and longitude, drop redundant columns
# origin_dest_locations = origin_dest_locations.dropna(subset=['Latitude', 'Longitude'])
origin_dest_locations = origin_dest_locations.drop(columns = ['Stanox'])
new_column_names = {'Latitude': 'Lat_DES', 'Longitude': 'Lon_DES', 'Station Name': 'STATION_DES'}
origin_dest_locations.rename(columns=new_column_names, inplace=True)


In [9]:
reasons_csv = pd.read_excel('https://sacuksprodnrdigital0001.blob.core.windows.net/historic-delay-attribution/Reference%20Files/Transpareny%20Page%20Attribution%20Glossary.xlsx', sheet_name='Incident Reason')[['Incident Reason', 'Incident Reason Description']]
reasons_csv.rename(columns={'Incident Reason': 'INCIDENT_REASON', 'Incident Reason Description': 'INCIDENT_REASON_DESCRIPTION'}, inplace=True)

In [10]:
EK_merged = origin_dest_locations.merge(reasons_csv, on='INCIDENT_REASON')
EK_merged.to_csv(f'../data/EK_all.csv', index=False)

In [11]:
EK_merged

,FINANCIAL_YEAR_AND_PERIOD,ORIGIN_DEPARTURE_DATE,TRUST_TRAIN_ID_AFFECTED,PLANNED_ORIG_LOC_CODE_AFF,PLANNED_ORIG_GBTT_DATETIME_AFF,PLANNED_ORIG_WTT_DATETIME_AFF,PLANNED_DEST_LOC_CODE_AFFECTED,PLANNED_DEST_GBTT_DATETIME_AFF,PLANNED_DEST_WTT_DATETIME_AFF,TRAIN_SERVICE_CODE_AFFECTED,SERVICE_GROUP_CODE_AFFECTED,OPERATOR_AFFECTED,ENGLISH_DAY_TYPE,APP_TIMETABLE_FLAG_AFF,TRAIN_SCHEDULE_TYPE_AFFECTED,TRACTION_TYPE_AFFECTED,TRAILING_LOAD_AFFECTED,TIMING_LOAD_AFFECTED,UNIT_CLASS_AFFECTED,INCIDENT_NUMBER,INCIDENT_CREATE_DATE,INCIDENT_START_DATETIME,INCIDENT_END_DATETIME,SECTION_CODE,NETWORK_RAIL_LOCATION_MANAGER,RESPONSIBLE_MANAGER,INCIDENT_REASON,ATTRIBUTION_STATUS,INCIDENT_EQUIPMENT,INCIDENT_DESCRIPTION,REACTIONARY_REASON_CODE,INCIDENT_RESPONSIBLE_TRAIN,PERFORMANCE_EVENT_CODE,START_STANOX,END_STANOX,EVENT_DATETIME,PFPI_MINUTES,RESP_TRAIN,REACT_TRAIN,Lat_OR,Lon_OR,STATION_OR,Lat_DES,Lon_DES,STATION_DES,INCIDENT_REASON_DESCRIPTION
0,2018/19_P01,2018-04-01,502V19MF01,50392.0,2018-04-01 09:54:00,2018-04-01 09:54:00,50401.0,2018-04-01 10:03:00,2018-04-01 10:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 09:54:00,0.0,NaN,NaN,51.559108,0.250912,Upminster Rail Station,51.559108,0.250912,Upminster Rail Station,System generated cancellation (NOT to be attri...
1,2018/19_P01,2018-04-01,502V36MO01,50401.0,2018-04-01 14:11:00,2018-04-01 14:11:00,50392.0,2018-04-01 14:20:00,2018-04-01 14:20:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50401.0,50401.0,2018-04-01 14:11:00,0.0,NaN,NaN,51.574640,0.182370,ROMFORD,51.574640,0.182370,ROMFORD,System generated cancellation (NOT to be attri...
2,2018/19_P01,2018-04-01,502V59MY01,50392.0,2018-04-01 19:54:00,2018-04-01 19:54:00,50401.0,2018-04-01 20:03:00,2018-04-01 20:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 19:54:00,0.0,NaN,NaN,51.559108,0.250912,Upminster Rail Station,51.559108,0.250912,Upminster Rail Station,System generated cancellation (NOT to be attri...
3,2018/19_P01,2018-04-01,502V35MN01,50392.0,2018-04-01 13:54:00,2018-04-01 13:54:00,50401.0,2018-04-01 14:03:00,2018-04-01 14:03:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 13:54:00,0.0,NaN,NaN,51.559108,0.250912,Upminster Rail Station,51.559108,0.250912,Upminster Rail Station,System generated cancellation (NOT to be attri...
4,2018/19_P01,2018-04-01,502V49MU01,50392.0,2018-04-01 17:24:00,2018-04-01 17:24:00,50401.0,2018-04-01 17:33:00,2018-04-01 17:33:00,21237001,EK05,EK,SU,Y,(V)STP Cancellation,EMU,NaN,NaN,317.0,925037.0,2018-04-01,2018-03-31 00:00:00,2018-03-31 23:59:00,50328,OQHN,PQHY,PD,Attribution Agreed,NaN,PLANNED CANCELLATION 31/03/18,PD,NaN,S,50392.0,50392.0,2018-04-01 17:24:00,0.0,NaN,NaN,51.559108,0.250912,Upminster Rail Station,51.559108,0.250912,Upminster Rail Station,System generated cancellation (NOT to be attri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790290,2022/23_P11,2023-01-14,879D60MZ14,87651.0,2023-01-14 20:16:00,2023-01-14 20:16:00,52051.0,2023-01-14 21:08:00,2023-01-14 21:06:00,22218000,EK99,EK,SA,Y,LTP,EMU,NaN,NaN,378.0,203759.0,2023-01-14,2023-01-14 20:00:00,2023-01-15 04:25:00,63633,OQM6,OQBN,OF,Attribution Agreed,NaN,9T52 ETCS ZFD,TG,NaN,P,87603.0,87603.0,2023-01-14 2

In [12]:
EK_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 790295 entries, 0 to 790294
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   FINANCIAL_YEAR_AND_PERIOD       790295 non-null  object        
 1   ORIGIN_DEPARTURE_DATE           790295 non-null  datetime64[ns]
 2   TRUST_TRAIN_ID_AFFECTED         790295 non-null  object        
 3   PLANNED_ORIG_LOC_CODE_AFF       790295 non-null  float64       
 4   PLANNED_ORIG_GBTT_DATETIME_AFF  692043 non-null  datetime64[ns]
 5   PLANNED_ORIG_WTT_DATETIME_AFF   790295 non-null  datetime64[ns]
 6   PLANNED_DEST_LOC_CODE_AFFECTED  790295 non-null  float64       
 7   PLANNED_DEST_GBTT_DATETIME_AFF  692043 non-null  datetime64[ns]
 8   PLANNED_DEST_WTT_DATETIME_AFF   790295 non-null  datetime64[ns]
 9   TRAIN_SERVICE_CODE_AFFECTED     790295 non-null  int64         
 10  SERVICE_GROUP_CODE_AFFECTED     790295 non-null  object 

In [13]:
missing_stanox = pd.DataFrame(EK_merged[np.isnan(EK_merged['Lat_OR'])]['START_STANOX'].unique())

In [17]:
missing_stanox = pd.DataFrame(EK_merged[np.isnan(EK_merged['Lat_OR'])]['START_STANOX'].unique())
missing_stanox1 = pd.DataFrame(EK_merged[np.isnan(EK_merged['Lat_DES'])]['START_STANOX'].unique())
missing_stanox = pd.concat([missing_stanox, missing_stanox1]).rename(columns={0: 'Stannox'}).drop_duplicates()

In [18]:
missing_stanox

,Stannox
0,52049.0
1,87604.0
2,72027.0
3,51603.0
4,52038.0
...,...
27,73823.0
28,72111.0
2,52074.0
21,51937.0


In [19]:
cif_tiplocs = pd.read_csv('../mapping_data/cif_tiplocs.csv')
stanox_tiploc = missing_stanox.merge(cif_tiplocs, on='Stannox', how='left').sort_values(by='Stannox').drop_duplicates(subset='Stannox')

In [22]:
stanox_tiploc.sort_values(by='Tiploc')

,Stannox,CRS,Tiploc,Description
5,51725.0,NaN,BROXDPL,BROXBOURNE DOWN PASS LOOP
1,87604.0,NaN,CFTNRJN,CROFTON ROAD JN
6,50406.0,NaN,CHDWHTT,CHADWELL HEATH TURNBACK
22,52219.0,NaN,CHNELSJ,CHANNELSEA JN
15,51938.0,NaN,CLAPTNJ,CLAPTON JN
25,72427.0,NaN,CMDN202,CAMDEN ROAD JN SIGNAL 1202
33,72421.0,NaN,CMDNRDJ,CAMDEN ROAD JN
17,72431.0,NaN,CMDNRIJ,CAMDEN ROAD INCLINE JN
0,52049.0,XIC,CNNBELL,CANONBURY ELL
31,51937.0,XMI,COPRMLJ,COPPERMILL JUNCTION


In [23]:
east_north = pd.read_excel('../mapping_data/TIPLOC Eastings and Northings.xlsx')
east_north.rename(columns={'TIPLOC': 'Tiploc'}, inplace=True)

In [24]:
stan_EN = stanox_tiploc[['Stannox', 'Tiploc']].merge(east_north, on='Tiploc', how='left')

In [30]:
stan_EN.loc[0,'EASTING'], stan_EN.loc[0,'NORTHING'] = 547701,187615
stan_EN.loc[1,'EASTING'], stan_EN.loc[1,'NORTHING'] = 533637,188602
stan_EN.loc[2,'EASTING'], stan_EN.loc[2,'NORTHING'] = 533637,188602
stan_EN.loc[10,'EASTING'], stan_EN.loc[10,'NORTHING'] = 533570,184780

In [31]:
stan_EN

,Stannox,Tiploc,NAME,EASTING,NORTHING
0,50406.0,CHDWHTT,NaN,547701.0,187615.0
1,51556.0,STOTS16,NaN,533637.0,188602.0
2,51560.0,STOTS17,NaN,533637.0,188602.0
3,51603.0,EHMDCEJ,EAST HAM DEP COUNTRY E. JN,543490.0,184818.0
4,51725.0,BROXDPL,BROXBOURNE DOWN PASS LOOP,537442.0,207286.0
5,51935.0,TTNHMSJ,TOTTENHAM SOUTH JN,534516.0,188784.0
6,51937.0,COPRMLJ,COPPERMILL JUNCTION,535048.0,187930.0
7,51938.0,CLAPTNJ,CLAPTON JN,535210.0,187342.0
8,52038.0,LEAJ,LEA JUNCTION,537667.0,184575.0
9,52049.0,CNNBELL,CANONBURY ELL,532319.0,184999.0


In [32]:
import requests

def getcoor(row):
    print(row['EASTING'])
    url = f"https://api.getthedata.com/bng2latlong/{int(row['EASTING'])}/{int(row['NORTHING'])}"
    response = requests.get(url)
    print(response.json())
    return response.json() 

In [33]:
stan_EN_c = stan_EN.copy()
for i, row in stan_EN.iterrows():
    result = getcoor(row)
    stan_EN_c.loc[i,'lat'] = result['latitude']
    stan_EN_c.loc[i,'lon'] = result['longitude']

547701.0
{'status': 'ok', 'easting': 547701, 'northing': 187615, 'latitude': 51.56807, 'longitude': 0.12969}
533637.0
{'status': 'ok', 'easting': 533637, 'northing': 188602, 'latitude': 51.58044, 'longitude': -0.07272}
533637.0
{'status': 'ok', 'easting': 533637, 'northing': 188602, 'latitude': 51.58044, 'longitude': -0.07272}
543490.0
{'status': 'ok', 'easting': 543490, 'northing': 184818, 'latitude': 51.54402, 'longitude': 0.06784}
537442.0
{'status': 'ok', 'easting': 537442, 'northing': 207286, 'latitude': 51.74742, 'longitude': -0.0105}
534516.0
{'status': 'ok', 'easting': 534516, 'northing': 188784, 'latitude': 51.58187, 'longitude': -0.05998}
535048.0
{'status': 'ok', 'easting': 535048, 'northing': 187930, 'latitude': 51.57407, 'longitude': -0.05263}
535210.0
{'status': 'ok', 'easting': 535210, 'northing': 187342, 'latitude': 51.56874, 'longitude': -0.05052}
537667.0
{'status': 'ok', 'easting': 537667, 'northing': 184575, 'latitude': 51.54329, 'longitude': -0.01617}
532319.0
{'st

In [34]:
output = stan_EN_c[['Stannox', 'lat', 'lon', 'NAME']]
output['Stannox'] = output['Stannox'].astype('int')
output.rename(columns={'Stannox':'Stanox', 'lat': 'Latitude', 'lon': 'Longitude', 'NAME': 'Station Name'}, inplace=True)

/tmp/ipykernel_8232/2329810162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['Stannox'] = output['Stannox'].astype('int')
/tmp/ipykernel_8232/2329810162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output.rename(columns={'Stannox':'Stanox', 'lat': 'Latitude', 'lon': 'Longitude', 'NAME': 'Station Name'}, inplace=True)


In [36]:
lookup_csv_path = "../mapping_data/stanox-lookup-lc.csv"
mapping_df = pd.read_csv(lookup_csv_path)
# mapping_df = mapping_df[['Stanox', 'Latitude', 'Longitude', 'Station Name']]
mapping_df

,Stanox,Latitude,Longitude,Station Name
0,50334,51.581910,0.205960,NaN
1,50337,51.584620,0.212550,GIDEA PARK MIDDLE SIDING
2,50401,51.574640,0.182370,ROMFORD
3,50422,51.561290,0.079170,ILFORD E.M.U.D.
4,50423,51.559000,0.068970,ILFORD
...,...,...,...,...
9719,87601,51.505064,-0.086064,London Bridge Rail Station
9720,87214,51.486189,-0.122857,Vauxhall Rail Station
9721,87201,51.495257,-0.144527,London Victoria Rail Station
9722,86005,51.361768,-0.457711,Weybridge Rail Station


In [37]:
out_df = pd.concat([output, mapping_df])
out_df

,Stanox,Latitude,Longitude,Station Name
0,50406,51.568070,0.129690,NaN
1,51556,51.580440,-0.072720,NaN
2,51560,51.580440,-0.072720,NaN
3,51603,51.544020,0.067840,EAST HAM DEP COUNTRY E. JN
4,51725,51.747420,-0.010500,BROXBOURNE DOWN PASS LOOP
...,...,...,...,...
9719,87601,51.505064,-0.086064,London Bridge Rail Station
9720,87214,51.486189,-0.122857,Vauxhall Rail Station
9721,87201,51.495257,-0.144527,London Victoria Rail Station
9722,86005,51.361768,-0.457711,Weybridge Rail Station


In [39]:
out_df.drop_duplicates(subset='Stanox').to_csv('../mapping_data/stanox-lookup-lc.csv', index=False)